In [115]:
import pickle
import numpy as np
np.set_printoptions(suppress=True)

In [116]:
saved_info = '../data/hmm.pkl'

In [117]:
with open(saved_info, 'rb') as fin:
    hmm = pickle.load(fin)

In [118]:
hmm

(array([3, 2, 2, 2, 0]),
 array([0.45394818, 0.21555641, 0.33049541]),
 array([[0.4213136 , 0.46357142, 0.11511499],
        [0.26773101, 0.41561989, 0.3166491 ],
        [0.4621726 , 0.28426716, 0.25356024]]),
 array([[0.30918247, 0.45485864, 0.46252636, 0.36956665],
        [0.34031305, 0.51317086, 0.37998398, 0.35372579],
        [0.35050448, 0.0319705 , 0.15748966, 0.27670756]]),
 3,
 4,
 5)

In [119]:
O,pi,a,b,N,M,T = hmm

In [120]:
print(N, M, T, type(N))

3 4 5 <class 'int'>


In [121]:
def forward():
    dp = np.zeros((N, T))
    for s in range(N):
        dp[s, 0] = pi[s]*b[s, O[0]]
    print(dp)
    # 先遍历 t，后遍历 s
    for t in range(1, T):
        for s in range(N):
            for s_prime in range(N):
                dp[s][t] += dp[s_prime][t-1]*a[s_prime][s]*b[s, O[t]]
    print(dp)
    return dp[:, -1].sum()
forward()

[[0.16776411 0.         0.         0.         0.        ]
 [0.07624786 0.         0.         0.         0.        ]
 [0.09145058 0.         0.         0.         0.        ]]
[[0.16776411 0.06168306 0.0206376  0.00739148 0.00178391]
 [0.07624786 0.05147155 0.02012802 0.00725743 0.00234173]
 [0.09145058 0.01049577 0.00410424 0.00154181 0.00124074]]


0.005366372712466162

In [122]:
def viterbi():
    dp = np.zeros((N, T))
    # 保存各个时刻从哪个状态转移到当前状态后发射出观测的概率最大
    records = np.zeros((N, T), dtype=np.int32)
    for s in range(N):
        dp[s, 0] = pi[s]*b[s, O[0]]
        records[s, 0] = s
    print(dp)
    for t in range(1, T):
        for s in range(N):
            temp = np.zeros(N)
            for s_prime in range(N):
                temp[s_prime] = dp[s_prime][t-1]*a[s_prime][s]*b[s, O[t]]
            records[s, t] = temp.argmax()
            dp[s, t] = temp.max()
    print(dp)
    print(records)
    # 最后时刻还没有找出哪个最大
    final_argmax = dp[:, -1].argmax()
    print(final_argmax)
    # 回溯
    res = []
    max_state = final_argmax
    for t in reversed(range(T)):
        res.append(max_state)
        max_state = records[max_state, t]
    return list(reversed(res))
viterbi()

[[0.16776411 0.         0.         0.         0.        ]
 [0.07624786 0.         0.         0.         0.        ]
 [0.09145058 0.         0.         0.         0.        ]]
[[0.16776411 0.03269196 0.00637064 0.00124144 0.00016171]
 [0.07624786 0.0295516  0.00575868 0.00112219 0.00019585]
 [0.09145058 0.0038024  0.00147371 0.00028718 0.00012455]]
[[0 0 0 0 0]
 [1 0 0 0 0]
 [2 1 1 1 1]]
1


[0, 0, 0, 0, 1]